## OpenAI 의 chat.completions.create 
- chat.completions.create 는 단순하게 메시지를 받아서 답변을 주는 것 보다 훨씬 더 많은 것을 할 수 있음.
- 챗봇모드, 스트리밍 모드, 이미지모드, 함수모드(tools) etc
    - 챗봇 모드 : 단순한 챗봇 -> 답변이 완성된 후에만 display
    - 스트리밍 모드 : 답변을 display
    - 이미지 모드 : image file을 사용할 경우 base64로 인코딩해서 url로 변경 가능
    - 함수모드 : 함수를 알아서 써줌. (사용할지 말지를 llm이 결정) 
- API Reference: https://platform.openai.com/docs/api-reference/chat


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

print("openai_key: " + os.getenv("OPENAI_API_KEY"))

In [7]:
import openai

openai.__version__

'1.34.0'

## Client 생성

- `client` 는 OpenAI 모듈로 생성된 인스턴스 이다.


In [8]:
from openai import OpenAI

client = OpenAI()

## chat.completions.create 



**주요 파라미터**

- `messages`: 지금까지의 대화를 구성하는 메시지 목록입니다.(mandatory)

- `model`: 모델id. "gpt-4o-mini", "gpt-4-turbo", gpt-3.5-turbo" - mini가 제일 쌈.

- `frequency_penalty`: -2.0에서 2.0 사이의 숫자. 양수 값은 지금까지 텍스트에 나타난 기존 빈도에 따라 새로운 토큰에 불이익을 주어 모델이 같은 줄을 그대로 반복할 가능성을 낮춥니다.

- `max_tokens`: 생성할 수 있는 최대 토큰 수입니다. 입력 토큰과 생성된 토큰의 총 길이는 모델의 컨텍스트 길이에 의해 제한됩니다.

- `n`: 각 입력 메시지에 대해 생성할 선택지(choices) 수입니다. [주의] 모든 선택 항목에서 생성된 토큰 수에 따라 요금이 부과된다는 점에 유의하세요. 비용을 최소화하려면 n을 1로 유지하세요.

- `presence_penalty`: -2.0에서 2.0 사이의 숫자. 값이 양수이면 지금까지 텍스트에 등장한 토큰에 따라 새로운 토큰에 불이익을 주므로 모델이 새로운 주제에 대해 이야기할 가능성이 높아집니다.

- `response_format`: 모델이 출력해야 하는 형식을 지정하는 객체입니다.  `{ "type": "json_object" }` 로 설정하면 JSON 모드가 활성화되어 json 형태로 답변. 답변이 json 형태로 되면, pandas, excel, csv 파일형태로 쉽게 변형 가능해서, 추후에 유용하게 사용될 수 있음. 

- `seed`: 이 기능을 지정하면 시스템이 결정론적으로 샘플링하여 동일한 시드와 매개변수를 사용한 반복 요청이 동일한 결과를 반환하도록 최선을 다할 것입니다. 결정론은 보장되지 않으며, `system_fingerprint` 응답 매개변수를 참조하여 백엔드의 변경 사항을 모니터링해야 합니다.

- `temperature`: 0에서 1 사이에서 사용할 샘플링 온도입니다. 0.8과 같이 값이 높으면 출력이 더 무작위적이고, 0.2와 같이 값이 낮으면 더 집중적이고 결정론적인 출력이 됩니다. 일반적으로 이 값이나 `top_p` 중 하나만 변경하는 것이 좋지만 둘 다 변경하지는 않는 것이 좋습니다.

- `top_p`: `temperature` 를 이용한 샘플링의 대안으로, 핵 샘플링이라고 하며, 모델이 `top_p` 확률을 가진 토큰의 결과를 고려하는 방식입니다. 따라서 0.1은 상위 10% 확률을 구성하는 토큰만 고려한다는 의미입니다. 일반적으로 이 값이나 `temperature` 중 하나를 변경하는 것이 좋지만 둘 다 변경하는 것은 권장하지 않습니다.


In [9]:
# chat.completions.create의 단순 챗봇 모드


def ask(question, content_only=True):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "당신은 친절한 AI Assistance 입니다.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        temperature=0.5,
    )
    # return completion.choices[0].message.content
    if content_only:
        return completion.choices[0].message.content
    else:
        return completion

## chat.completions.create 단순 챗봇 모드의 response format
- response는 json format

```json
{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-4o-mini",
  "system_fingerprint": "fp_44709d6fcb",
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "\n\nHello there, how may I assist you today?",
    },
    "logprobs": null,
    "finish_reason": "stop"
  }],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}
```

In [10]:
# chat.completions.create의 대화형 챗봇 모드의 결과를 모두 출력
completion = ask("파리 올림픽에 대해 아세요?", False)
print(completion)

print()
completion = ask("파리 올림픽에 대해 아세요?", True)
print(completion)

ChatCompletion(id='chatcmpl-9r1gr8kAOTQR9YaLQpWd5ZJmHNWKf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='네, 2024년 파리 올림픽은 제33회 하계 올림픽으로, 프랑스 파리에서 2024년 7월 26일부터 8월 11일까지 개최될 예정입니다. 이번 올림픽은 파리가 1924년에 이어 100년 만에 두 번째로 개최하는 올림픽입니다.\n\n파리 올림픽은 다양한 스포츠 종목이 포함되며, 새로운 종목으로는 스케이트보드, 스포츠 클라이밍, 서핑 등이 추가되었습니다. 또한, 지속 가능성과 친환경적인 방향을 강조하여, 많은 경기 시설이 기존의 시설을 재활용하거나 임시 구조물로 건설될 예정입니다.\n\n올림픽을 통해 전 세계의 선수들이 모여 경쟁하고, 문화 교류의 장이 열리는 만큼 많은 기대를 받고 있습니다. 추가로 궁금한 점이 있다면 말씀해 주세요!', role='assistant', function_call=None, tool_calls=None))], created=1722424909, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_611b667b19', usage=CompletionUsage(completion_tokens=192, prompt_tokens=31, total_tokens=223))

네, 파리 올림픽은 2024년 여름 올림픽으로, 프랑스 파리에서 개최됩니다. 이 대회는 2024년 7월 26일부터 8월 11일까지 진행될 예정입니다. 파리 올림픽은 1900년과 1924년에 이어 세 번째로 파리에서 열리는 올림픽입니다.

이번 대회는 여러 스포츠 종목에서 전 세계 선수들이 경쟁하며, 특히 지속 가능성과 환경 보호를 강조하는 여러 계획이 포함되어 있습니다. 또한, 다양한 문화 행사와 프

In [12]:
print(ask("한국의 수도는 어디인가요?"))
print(ask("한국의 대표 아이돌은 누구인가요?"))
print(ask("한국경제의 미래에 대해서 이야기해주세요?"))

한국의 수도는 서울입니다.
한국의 대표 아이돌은 여러 명이 있지만, 특히 BTS(방탄소년단)와 BLACKPINK가 세계적으로 유명합니다. BTS는 다양한 음악 스타일과 강력한 메시지로 많은 사랑을 받고 있으며, BLACKPINK는 강렬한 퍼포먼스와 독특한 음악으로 글로벌 팬층을 형성하고 있습니다. 그 외에도 EXO, TWICE, Red Velvet 등 많은 인기 아이돌 그룹이 있습니다. 각 그룹마다 독특한 매력과 팬층이 있어 한국 아이돌 문화의 다양성을 보여줍니다.
한국 경제의 미래에 대한 전망은 여러 요소에 따라 달라질 수 있습니다. 다음은 몇 가지 주요 요인과 그에 따른 가능성입니다.

1. **기술 혁신**: 한국은 반도체, 전자, 자동차 등 다양한 분야에서 기술 혁신을 선도하고 있습니다. 특히 인공지능(AI), 바이오 헬스, 친환경 기술 등의 분야에서 성장이 기대됩니다. 이러한 혁신이 지속된다면 한국 경제는 더욱 경쟁력을 갖출 수 있습니다.

2. **인구 구조 변화**: 한국은 고령화 사회로 접어들고 있으며, 이는 노동력 감소와 소비 패턴 변화로 이어질 수 있습니다. 이러한 인구 구조 변화에 대응하기 위한 정책과 전략이 필요합니다.

3. **글로벌 경제와의 연계**: 한국은 수출 중심의 경제 구조를 가지고 있기 때문에 글로벌 경제 상황에 큰 영향을 받습니다. 특히 미국, 중국과의 관계가 중요하며, 이를 통해 새로운 시장을 개척하고 무역을 확대할 수 있는 기회가 있습니다.

4. **지속 가능한 발전**: 환경 문제에 대한 인식이 높아짐에 따라 지속 가능한 발전을 위한 노력이 필요합니다. 그린 뉴딜 정책과 같은 친환경 정책이 경제 성장과 환경 보호를 동시에 추구하는 방향으로 나아갈 수 있습니다.

5. **정치적 안정성 및 정책**: 정치적 안정성과 효율적인 경제 정책은 경제 성장에 중요한 요소입니다. 정부의 정책 방향과 실행력이 한국 경제의 미래에 큰 영향을 미칠 것입니다.

6. **사회적 불평등**: 경제 성장 속에서 사회적 불평등 문제가 대두되

## 스트리밍(Streaming) 모드
<span style="color: red;"> 이때, create() 함수내에 `stream=True` 옵션을 지정하면 됩니다.</span>

- 토큰 단위로 실시간 출력을 위해서는 `completion` 을 순회하면서, `choices.delta.content` 를 출력해야 합니다.
- 여기서 `choices.delta.content` 란 ?  
    - OpenAI API의 스트리밍 모드 응답에서 실시간으로 전달되는 텍스트 조각을 나타냅니다.
    
- streaming mode의 response 형태
```json
{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-4o-mini", "system_fingerprint": "fp_44709d6fcb", 
    "choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-4o-mini", "system_fingerprint": "fp_44709d6fcb", 
    "choices":[{"index":0,"delta":{"content":"Hello"},"logprobs":null,"finish_reason":null}]}
....
{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-4o-mini", "system_fingerprint": "fp_44709d6fcb", 
    "choices":[{"index":0,"delta":{},"logprobs":null,"finish_reason":"stop"}]}
```


In [13]:
# chat.completions.create의 streaming 모드

from openai import OpenAI


def ask_stream(question):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "당신은 친절한 AI Assistance 입니다.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        stream=True,  # 스트림 모드 활성화
    )
    return completion

In [14]:
# 스트림 모드에서는 completion.choices 를 반복문으로 순회
question = "한국에서 가장 인기 있는 아이돌 3가지에 대해 알려줘. 각각 하나의 섹션으로 따로따로 이야기해주세요. 마지막에 이중에서 너가 좋아하는 아이돌 1가지만 말해줘."
completion = ask_stream(question)
print(completion)
print("-----------------")

final_answer = []
# delta.content를 사용해서 chunk의 내용을 가져옴. why? 하나하나씩 출력위해
for chunk in completion:
    # chunk 를 저장
    chunk_content = chunk.choices[0].delta.content
    # chunk 가 문자열이면 final_answer 에 추가
    if isinstance(chunk_content, str):
        final_answer.append(chunk_content)
        # 토큰 단위로 실시간 답변 출력
        print(chunk_content, end="")

-----------------
물론입니다! 한국에서 인기 있는 아이돌을 세 가지 섹션으로 나누어 각각 설명해드리겠습니다.

### 1. BTS (방탄소년단)
BTS는 2013년에 데뷔한 보이 그룹으로, 전 세계적으로 팬층을 형성하고 있는 K-pop의 대표주자입니다. 그들의 음악은 힙합, 팝, R&B 등을 아우르며, 사회적 메시지를 담고 있는 가사로도 유명합니다. BTS는 다양한 awards를 수상하며, 세계 투어를 통해 많은 팬들을 만나기도 했습니다. 그들의 팬층은 '아미'로 알려져 있으며, 이들은 BTS의 활동을 지지하고 사랑하는 열정적인 커뮤니티를 이루고 있습니다.

### 2. BLACKPINK
BLACKPINK는 2016년에 데뷔한 걸 그룹으로, 글로벌 K-pop 열풍을 주도하고 있습니다. 그들의 음악은 강렬한 비트와 중독성 있는 멜로디로 사랑받고 있으며, 'DDU-DU DDU-DU', 'How You Like That' 등 많은 히트곡이 있습니다. BLACKPINK는 또한 독특한 패션 감각과 시선을 사로잡는 뮤직 비디오로 유명하며, 팬층은 '블링크'로 알려져 있습니다. 그들은 미국, 유럽, 아시아 등지에서 활발한 활동을 이어가고 있습니다.

### 3. EXO
EXO는 2012년에 데뷔한 보이 그룹으로, 뛰어난 실력과 다채로운 앨범으로 많은 사랑을 받고 있습니다. 그들의 음악은 팝, R&B, EDM 등 다양한 장르를 탐구하며, 하모니와 감정 표현에 탁월합니다. EXO의 앨범은 다수의 차트에서 1위를 기록하며, 팬들은 '엑소-L'로 불립니다. EXO는 강력한 무대 퍼포먼스와 연기, 예능 등 다양한 분야에서 활동하며 많은 팬들에게 사랑받고 있습니다.

이 세 그룹 중에서 제가 개인적으로 가장 좋아하는 아이돌은 **BTS**입니다. 그들의 음악과 메시지가 많은 사람들에게 긍정적인 영향을 주는 것 같아서요.

In [15]:
print(final_answer)

# 답변은 사실 토큰단위로 나눠져있음.

['', '물', '론', '입니다', '!', ' 한국', '에서', ' 인기', ' 있는', ' 아이', '돌', '을', ' 세', ' 가지', ' 섹', '션', '으로', ' 나', '누', '어', ' 각각', ' 설명', '해', '드', '리', '겠습니다', '.\n\n', '###', ' ', '1', '.', ' BTS', ' (', '방', '탄', '소년', '단', ')\n', 'B', 'TS', '는', ' ', '201', '3', '년에', ' 데', '뷔', '한', ' 보', '이', ' 그룹', '으로', ',', ' 전', ' 세계', '적으로', ' 팬', '층', '을', ' 형', '성', '하고', ' 있는', ' K', '-pop', '의', ' 대표', '주', '자', '입니다', '.', ' 그', '들의', ' 음악', '은', ' 힙', '합', ',', ' 팝', ',', ' R', '&B', ' 등을', ' 아', '우', '르', '며', ',', ' 사회', '적', ' 메시', '지를', ' 담', '고', ' 있는', ' 가', '사', '로', '도', ' 유명', '합니다', '.', ' BTS', '는', ' 다양한', ' awards', '를', ' 수', '상', '하며', ',', ' 세계', ' 투', '어', '를', ' 통해', ' 많은', ' 팬', '들을', ' 만나', '기도', ' 했', '습니다', '.', ' 그', '들의', ' 팬', '층', '은', " '", '아', '미', "'", '로', ' 알려', '져', ' 있으며', ',', ' 이', '들은', ' BTS', '의', ' 활동', '을', ' 지', '지', '하고', ' 사랑', '하는', ' 열', '정', '적인', ' 커', '뮤니', '티', '를', ' 이루', '고', ' 있습니다', '.\n\n', '###', ' ', '2', '.', ' BLACK', 'P', 'INK', '\n'

In [16]:
# 전체 답변인 final_answer 를 문자열로 변환하여 출력
final_answer = "".join(final_answer)
print(final_answer)

물론입니다! 한국에서 인기 있는 아이돌을 세 가지 섹션으로 나누어 각각 설명해드리겠습니다.

### 1. BTS (방탄소년단)
BTS는 2013년에 데뷔한 보이 그룹으로, 전 세계적으로 팬층을 형성하고 있는 K-pop의 대표주자입니다. 그들의 음악은 힙합, 팝, R&B 등을 아우르며, 사회적 메시지를 담고 있는 가사로도 유명합니다. BTS는 다양한 awards를 수상하며, 세계 투어를 통해 많은 팬들을 만나기도 했습니다. 그들의 팬층은 '아미'로 알려져 있으며, 이들은 BTS의 활동을 지지하고 사랑하는 열정적인 커뮤니티를 이루고 있습니다.

### 2. BLACKPINK
BLACKPINK는 2016년에 데뷔한 걸 그룹으로, 글로벌 K-pop 열풍을 주도하고 있습니다. 그들의 음악은 강렬한 비트와 중독성 있는 멜로디로 사랑받고 있으며, 'DDU-DU DDU-DU', 'How You Like That' 등 많은 히트곡이 있습니다. BLACKPINK는 또한 독특한 패션 감각과 시선을 사로잡는 뮤직 비디오로 유명하며, 팬층은 '블링크'로 알려져 있습니다. 그들은 미국, 유럽, 아시아 등지에서 활발한 활동을 이어가고 있습니다.

### 3. EXO
EXO는 2012년에 데뷔한 보이 그룹으로, 뛰어난 실력과 다채로운 앨범으로 많은 사랑을 받고 있습니다. 그들의 음악은 팝, R&B, EDM 등 다양한 장르를 탐구하며, 하모니와 감정 표현에 탁월합니다. EXO의 앨범은 다수의 차트에서 1위를 기록하며, 팬들은 '엑소-L'로 불립니다. EXO는 강력한 무대 퍼포먼스와 연기, 예능 등 다양한 분야에서 활동하며 많은 팬들에게 사랑받고 있습니다.

이 세 그룹 중에서 제가 개인적으로 가장 좋아하는 아이돌은 **BTS**입니다. 그들의 음악과 메시지가 많은 사람들에게 긍정적인 영향을 주는 것 같아서요.


## 대화의 기억


`client.chat.completions.create()` 함수는 자체적으로 `대화를 저장하는 기능이 없다`.

=> 이전 문맥(context)을 기억하면서 대화를 이어나가기 위해서는(일반적인 ChatBot 처럼) 다음과 같이  ``messages` 옵션에 메시지를 추가해야 함`

messages는 대화의 각 부분을 구성하는 데 사용되며, `각 메시지는 "role"과 "content"라는 두 가지 주요 요소를 포함하는 딕셔너리 형태`로 되어 있음. 

각 "role"의 의미:

- "system": 시스템 전역 설정. 예를 들어, `모델에 특정한 페르소나(persona)`를 부여하거나, `대화의 맥락을 설정`하는 데 사용. chatbot의 custom instruction과 같음
- "user": 사용자의 입력. 이는 대화에서 사용자가 질문하거나 `요청한 내용`임.
- "assistant": AI 모델(예: ChatGPT)의 응답. 


In [17]:
def ask(question):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. You must answer in Korean.",  # 한국어로 답변해야 한다는 role
            },
            {
                "role": "user",
                "content": question,  # 사용자의 질문을 입력
            },
        ],
    )
    # 답변을 반환
    return completion.choices[0].message.content

In [18]:
# 첫 번째 질문
ask("창원에 대해서 설명해주세요")

"창원은 대한민국 경상남도에 위치한 도시로, 경상남도의 도청 소재지입니다. 이 도시는 경남의 중심부에 위치해 있으며, 산업 발전과 도시화가 활발하게 이루어진 지역입니다. 창원은 주로 기계, 자동차, 전자, 조선업 등 다양한 산업이 발달해 있어서 산업 도시로 유명합니다.\n\n창원시는 또한 자연 환경과 문화 유산이 잘 어우러져 있는 곳입니다. 창원 원형 관광지로 유명한 '창원시립미술관', '마산어명'과 같은 역사적 명소들과 아름다운 공원들이 있습니다. 그 중 '팔용산'과 '상복산'은 하이킹을 즐기는 사람들에게 인기 있는 장소입니다.\n\n또한, 창원은 교육과 연구 개발에도 힘을 쏟고 있어 여러 대학과 연구소들이 위치하고 있습니다. 지역 주민들이 자부심을 느낄 수 있는 다양한 문화 행사와 축제도 정기적으로 열립니다.\n\n교통 편의성도 좋아 전국적으로 접근성이 뛰어나며, KTX와 고속도로가 지나가는 지리적 이점이 있습니다. 이러한 점들을 통해 창원은 대한민국 내에서 중요한 산업 및 문화 중심지로 자리잡고 있습니다."

In [19]:
# 두 번째 질문
ask("거기를 가장 빠르게 가는 방법은 무엇인가요?")

'거기를 가장 빠르게 가는 방법은 여러 가지가 있을 수 있습니다. 대중교통을 이용할 경우, 가장 빠른 노선이나 시간표를 확인하는 것이 좋습니다. 자동차를 이용할 경우, 네비게이션 앱을 사용하여 실시간 교통 상황을 고려한 경로를 검색하는 것이 효과적입니다. 자전거나 도보로 이동할 경우, 직진하는 경로를 선택하고, 이동하는 시간대에 따른 교통량도 감안해야 합니다. 구체적인 장소를 알려주시면 더욱 정확한 정보를 제공해 드릴 수 있습니다.'

이전 질문을 기억하지 못함. 
이를 해결하기 위해, 대화의 각 부분(사용자의 질문과 AI의 응답)을 messages 리스트에 순차적으로 추가해서 기억하게끔 하자요

일단 전역변수 써서 쉽게 구현후, 나중에 깔끔하게 구현해보겠음.


In [21]:
message_history = []


def ask(question):
    if len(message_history) == 0:
        # 최초 질문
        message_history.append(
            {
                "role": "system",
                "content": "You are a helpful assistant. You must answer in Korean.",
            }
        )

    # 사용자 질문 추가
    message_history.append(
        {
            "role": "user",
            "content": question,
        },
    )

    # GPT에 질문을 전달하여 답변을 생성
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_history,
    )

    # 사용자 질문에 대한 답변을 추가
    message_history.append(
        {
            "role": "assistant",  # 답변을 한 역할은 assistant
            "content": completion.choices[0].message.content,
        }
    )

    return completion.choices[0].message.content

In [22]:
# 최초 질문
print(ask("창원에 대해서 설명해주세요"))

창원은 대한민국 경상남도에 위치한 도시로, 경남의 도청소재지입니다. 창원은 2010년까지는 창원시, 마산시, 진해시가 통합되어 현재의 창원시가 형성되었습니다. 이는 행정 효율성을 높이고 지역 발전을 도모하기 위한 조치였습니다.

창원은 산업 도시로서 특히 조선업, 기계, 전자, 자동차 등의 제조업이 발달해 있습니다. 삼성전자와 현대자동차와 같은 대기업의 생산 공장이 위치하고 있어 경제적으로 중요한 지역입니다.

자연환경적으로는 다대포 해변, 진해 군항제와 같은 관광 명소가 있으며, 진해는 벚꽃으로 유명하여 매년 많은 관광객이 방문합니다. 또한, 창원에는 다양한 문화시설과 공원이 있어 시민들이 여가를 즐길 수 있는 공간이 많이 마련되어 있습니다.

창원시는 또한 교육과 연구 개발에도 중점을 두고 있으며, 여러 대학과 연구기관이 위치해 있어 교육 인프라가 잘 갖추어져 있습니다. 이러한 요소들로 인해 창원은 산업과 교육, 문화가 조화롭게 발전하는 도시로 평가받고 있습니다.


In [23]:
# 두 번째 질문
print(ask("거기를 가장 빠르게 가는 방법은 무엇인가요?"))

창원으로 가는 가장 빠른 방법은 출발 지점에 따라 다릅니다. 일반적인 경우, 다음과 같은 방법들이 있습니다:

1. **자동차 이용**: 자가용을 이용하면 최단 경로를 선택할 수 있어 빠르게 이동할 수 있습니다. 주요 고속도로와 국도를 이용해 접근하면 됩니다.

2. **KTX(고속철도) 이용**: 서울이나 다른 대도시에서 출발하는 경우 KTX를 이용하면 빠르게 이동할 수 있습니다. KTX가 창원까지 연결되는 경우에는 가장 편리한 방법 중 하나입니다.

3. **시외버스 이용**: 서울이나 대구 등 주요 도시에서 창원으로 가는 시외버스도 있으며, 이 방법도 비교적 빠르고 편리합니다.

4. **비행기**: 먼 거리에서 창원으로 이동하는 경우, 가까운 공항으로 비행기를 이용한 후 내륙 교통으로 창원에 도착할 수도 있습니다.

출발 지점, 상황에 따라 가장 빠른 방법을 선택하면 됩니다. 특히 교통 상황을 고려하는 것이 중요합니다.


In [24]:
message_history

[{'role': 'system',
  'content': 'You are a helpful assistant. You must answer in Korean.'},
 {'role': 'user', 'content': '창원에 대해서 설명해주세요'},
 {'role': 'assistant',
  'content': '창원은 대한민국 경상남도에 위치한 도시로, 경남의 도청소재지입니다. 창원은 2010년까지는 창원시, 마산시, 진해시가 통합되어 현재의 창원시가 형성되었습니다. 이는 행정 효율성을 높이고 지역 발전을 도모하기 위한 조치였습니다.\n\n창원은 산업 도시로서 특히 조선업, 기계, 전자, 자동차 등의 제조업이 발달해 있습니다. 삼성전자와 현대자동차와 같은 대기업의 생산 공장이 위치하고 있어 경제적으로 중요한 지역입니다.\n\n자연환경적으로는 다대포 해변, 진해 군항제와 같은 관광 명소가 있으며, 진해는 벚꽃으로 유명하여 매년 많은 관광객이 방문합니다. 또한, 창원에는 다양한 문화시설과 공원이 있어 시민들이 여가를 즐길 수 있는 공간이 많이 마련되어 있습니다.\n\n창원시는 또한 교육과 연구 개발에도 중점을 두고 있으며, 여러 대학과 연구기관이 위치해 있어 교육 인프라가 잘 갖추어져 있습니다. 이러한 요소들로 인해 창원은 산업과 교육, 문화가 조화롭게 발전하는 도시로 평가받고 있습니다.'},
 {'role': 'user', 'content': '거기를 가장 빠르게 가는 방법은 무엇인가요?'},
 {'role': 'assistant',
  'content': '창원으로 가는 가장 빠른 방법은 출발 지점에 따라 다릅니다. 일반적인 경우, 다음과 같은 방법들이 있습니다:\n\n1. **자동차 이용**: 자가용을 이용하면 최단 경로를 선택할 수 있어 빠르게 이동할 수 있습니다. 주요 고속도로와 국도를 이용해 접근하면 됩니다.\n\n2. **KTX(고속철도) 이용**: 서울이나 다른 대도시에서 출발하는 경우 KTX를 이용하면 빠르게 이동할 수 있습니다. KTX가 창

In [25]:
print(ask("이전의 내용을 영어로 답변해주세요"))

The fastest way to get to Changwon depends on your starting point. Generally, the following options are available:

1. **By Car**: Using a personal vehicle allows you to choose the shortest route and reach your destination quickly. You can access major highways and national roads.

2. **Using KTX (Korea Train Express)**: If you are departing from Seoul or other major cities, taking the KTX is one of the fastest ways to get to Changwon, as it connects directly to the city.

3. **Intercity Bus**: Intercity buses are available from major cities like Seoul and Daegu to Changwon, making this a relatively fast and convenient option.

4. **By Plane**: If you are traveling from a distant location, you might take a flight to the nearest airport and then use local transportation to reach Changwon.

It's important to choose the fastest method based on your departure point and to consider traffic conditions as well.


In [26]:
message_history

[{'role': 'system',
  'content': 'You are a helpful assistant. You must answer in Korean.'},
 {'role': 'user', 'content': '창원에 대해서 설명해주세요'},
 {'role': 'assistant',
  'content': '창원은 대한민국 경상남도에 위치한 도시로, 경남의 도청소재지입니다. 창원은 2010년까지는 창원시, 마산시, 진해시가 통합되어 현재의 창원시가 형성되었습니다. 이는 행정 효율성을 높이고 지역 발전을 도모하기 위한 조치였습니다.\n\n창원은 산업 도시로서 특히 조선업, 기계, 전자, 자동차 등의 제조업이 발달해 있습니다. 삼성전자와 현대자동차와 같은 대기업의 생산 공장이 위치하고 있어 경제적으로 중요한 지역입니다.\n\n자연환경적으로는 다대포 해변, 진해 군항제와 같은 관광 명소가 있으며, 진해는 벚꽃으로 유명하여 매년 많은 관광객이 방문합니다. 또한, 창원에는 다양한 문화시설과 공원이 있어 시민들이 여가를 즐길 수 있는 공간이 많이 마련되어 있습니다.\n\n창원시는 또한 교육과 연구 개발에도 중점을 두고 있으며, 여러 대학과 연구기관이 위치해 있어 교육 인프라가 잘 갖추어져 있습니다. 이러한 요소들로 인해 창원은 산업과 교육, 문화가 조화롭게 발전하는 도시로 평가받고 있습니다.'},
 {'role': 'user', 'content': '거기를 가장 빠르게 가는 방법은 무엇인가요?'},
 {'role': 'assistant',
  'content': '창원으로 가는 가장 빠른 방법은 출발 지점에 따라 다릅니다. 일반적인 경우, 다음과 같은 방법들이 있습니다:\n\n1. **자동차 이용**: 자가용을 이용하면 최단 경로를 선택할 수 있어 빠르게 이동할 수 있습니다. 주요 고속도로와 국도를 이용해 접근하면 됩니다.\n\n2. **KTX(고속철도) 이용**: 서울이나 다른 대도시에서 출발하는 경우 KTX를 이용하면 빠르게 이동할 수 있습니다. KTX가 창

In [27]:
# 이 코드는 위의 것을 종합해서 대화 내용을 기억을 하는 챗봇을 만든 것임.  깖끔하게 구현 class로
from openai import OpenAI


class Chatbot:
    def __init__(self):
        self.client = OpenAI()
        self.message_history = [
            {
                "role": "system",
                "content": "You are a helpful assistant. You must answer in Korean.",
            }
        ]

    def ask(self, question):
        # 사용자 질문 추가
        self.message_history.append(
            {
                "role": "user",
                "content": question,
            }
        )

        # GPT에 질문을 전달하여 답변을 생성
        completion = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.message_history,
        )

        # 사용자 질문에 대한 답변을 추가
        assistant_response = completion.choices[0].message.content
        self.message_history.append(
            {"role": "assistant", "content": assistant_response}
        )

        return assistant_response


# 챗봇 인스턴스 생성
chatbot = Chatbot()

while True:
    user_input = input("질문을 입력하세요 (종료하려면 'q' 입력): ")
    if user_input.lower() == "q":
        print("챗봇을 종료합니다. 감사합니다.")
        break

    response = chatbot.ask(user_input)
    print("You      :", user_input)
    print("Assistant:", response)
    print(" ")

You      : 창원
Assistant: 창원은 대한민국 경상남도에 위치한 도시로, 경상남도의 도청 소재지입니다. 창원은 산업도시로 유명하며, 특히 기계, 조선, 전자 관련 산업이 발달해 있습니다. 또한, 자연 경관이 아름답고 다양한 관광지가 있어 많은 사람들이 찾는 곳입니다. 유명한 관광지로는 용지호수, 마산항, 창원시립미술관 등이 있습니다. 창원에 대해 더 알고 싶은 정보가 있나요?
 
You      : 서울에서 어떻게 가?
Assistant: 서울에서 창원으로 가는 방법은 여러 가지가 있습니다. 아래에 몇 가지 주요한 방법을 소개해 드릴게요.

1. **기차 (KTX)**:
   - 서울역에서 KTX를 이용하여 창원역으로 이동할 수 있습니다.
   - KTX는 약 3시간 정도 소요되며, 편리하고 빠른 방법입니다.

2. **버스**:
   - 서울의 여러 고속버스터미널(예: 서울고속버스터미널, 동서울종합버스터미널)에서 창원행 고속버스를 이용할 수 있습니다.
   - 버스는 약 4~5시간 정도 소요됩니다.

3. **자동차**:
   - 자가용을 이용하는 경우, 경부고속도로를 타고 가다가 밀양, 사천 방향으로 갈아타면 창원에 도착할 수 있습니다.
   - 소요 시간은 교통 상황에 따라 다르지만 보통 3~4시간 정도 걸립니다.

어떤 방법으로 가실 건지에 따라 선택하시면 좋을 것 같습니다! 추가적인 정보가 필요하시면 언제든지 말씀해 주세요.
 
챗봇을 종료합니다. 감사합니다.


## json_object 답변형식

이번에는 GPT 의 출력 형식을 지정하는 방법을 살펴보겠습니다. 다음의 예시는 프롬프트를 활용하여 답변을 JSON 형식으로 받는 예제 입니다.


In [33]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            # 답변 형식을 JSON 으로 받기 위해 프롬프트에 JSON 형식을 지정
            "content": "You are a helpful assistant designed to output JSON. You must answer in Korean.",
        },
        {
            "role": "user",
            "content": "Claude에 대해서 설명해주세요",
        },
    ],
    response_format={
        "type": "json_object"
    },  # response_forma에 답변 형식을 JSON 으로 지정
)

print(completion.choices[0].message.content)

{
  "Claude": {
    "개요": {
      "정의": "Claude는 AI 언어 모델로, 인공지능 기법을 사용하여 자연어 처리를 수행합니다.",
      "개발자": "Anthropic",
      "출시년도": 2023
    },
    "특징": {
      "대화능력": "Claude는 자연스러운 대화를 할 수 있도록 설계되었습니다.",
      "안전성": "사용자의 안전과 프라이버시를 최우선으로 고려하여 개발되었습니다.",
      "다양한 언어 지원": "다양한 언어를 이해하고 생성할 수 있는 능력을 갖추고 있습니다."
    },
    "응용 분야": [
      "고객 서비스",
      "콘텐츠 생성",
      "언어 번역",
      "교육",
      "연구 지원"
    ],
    "관련성": {
      "비슷한 모델": [
        "ChatGPT",
        "BERT",
        "GPT-3"
      ],
      "차별점": "Claude는 인간과의 상호작용을 보다 안전하고 유익하게 만들기 위해 설계된 점에서 차별화됩니다."
    }
  }
}


In [34]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant designed to output JSON.",
        },
        {
            "role": "user",
            "content": "AI에 대해서 4지선다형 문제를 내주고, 정답 번호를 알려주세요.",
        },
    ],
    temperature=0.5,
    max_tokens=2024,
)
print(response.choices[0].message.content)

{"question": "AI의 주요 목적은 무엇인가요?", 
 "options": {
   "A": "인간의 감정을 완벽하게 이해하기 위해",
   "B": "자율적으로 문제를 해결하고 결정을 내리기 위해",
   "C": "모든 인간의 직업을 대체하기 위해",
   "D": "단순히 데이터를 저장하고 관리하기 위해"
 }, 
 "correct_answer": "B"}


`response_format={"type": "json_object"}` 으로 출력시 json 형태로 출력되는 것을 확인 가능했음.

`json` 형식으로 출력 값을 받으면 데이터베이스에 저장하거나 파일형태로 저장하는데 용이합니다.  json 말고도 다른 타입도 가능함.


In [35]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant designed to output JSON.",
        },
        {
            "role": "user",
            "content": """
                AI를 주제로 4지선다형 객관식 문제를 5개를 만들어주세요. 
                정답은 번호로 알려주세요. 
                난이도는 [상, 중, 하] 중 하나로 표기해 주세요.
            """,
        },
    ],
    temperature=0.5,
    max_tokens=2048,
)

In [36]:
workbook = response.choices[0].message.content
print(workbook)

{
  "questions": [
    {
      "question": "인공지능의 기본 개념 중 하나로, 기계가 인간의 지능적인 작업을 수행할 수 있도록 하는 기술은 무엇인가?",
      "options": [
        "A. 머신러닝",
        "B. 데이터베이스",
        "C. 클라우드 컴퓨팅",
        "D. 블록체인"
      ],
      "correct_answer": 1,
      "difficulty": "하"
    },
    {
      "question": "다음 중 인공지능의 주요 응용 분야가 아닌 것은 무엇인가?",
      "options": [
        "A. 자율주행차",
        "B. 이미지 인식",
        "C. 금융 거래",
        "D. 전통적인 농업"
      ],
      "correct_answer": 4,
      "difficulty": "중"
    },
    {
      "question": "인공지능의 발전에 기여한 알고리즘 중 하나로, 신경망을 기반으로 한 것은 무엇인가?",
      "options": [
        "A. 서포트 벡터 머신",
        "B. 결정 트리",
        "C. 딥러닝",
        "D. 군집화"
      ],
      "correct_answer": 3,
      "difficulty": "상"
    },
    {
      "question": "인공지능의 윤리적 문제 중 하나로, 알고리즘의 결정 과정이 불투명한 현상을 무엇이라고 하는가?",
      "options": [
        "A. 블랙박스 문제",
        "B. 데이터 편향",
        "C. 과적합",
        "D. 모델 일반화"
      ],
      "correct_answer": 1,
      "difficulty": "중"
    },
    {
      "q

5개의 결과 값을 출력합니다.


아래의 코드는 json 라이브러리를 사용하여 JSON 형식의 답변을 파이썬 객체로 변환하는 코드입니다.


In [37]:
import json
import pandas as pd


# JSON 문자열을 Python 딕셔너리로 변환
answers = json.loads(workbook)

# 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(answers["questions"])

# options 열을 개별 열로 분리
df = pd.concat([df.drop(["options"], axis=1), df["options"].apply(pd.Series)], axis=1)

# 열 이름 변경
df.columns = ["question", "정답", "난이도", "옵션1", "옵션2", "옵션3", "옵션4"]

In [38]:
df.head()

,question,정답,난이도,옵션1,옵션2,옵션3,옵션4
0,"인공지능의 기본 개념 중 하나로, 기계가 인간의 지능적인 작업을 수행할 수 있도록 ...",1,하,A. 머신러닝,B. 데이터베이스,C. 클라우드 컴퓨팅,D. 블록체인
1,다음 중 인공지능의 주요 응용 분야가 아닌 것은 무엇인가?,4,중,A. 자율주행차,B. 이미지 인식,C. 금융 거래,D. 전통적인 농업
2,"인공지능의 발전에 기여한 알고리즘 중 하나로, 신경망을 기반으로 한 것은 무엇인가?",3,상,A. 서포트 벡터 머신,B. 결정 트리,C. 딥러닝,D. 군집화
3,"인공지능의 윤리적 문제 중 하나로, 알고리즘의 결정 과정이 불투명한 현상을 무엇이라...",1,중,A. 블랙박스 문제,B. 데이터 편향,C. 과적합,D. 모델 일반화
4,다음 중 인공지능 기술이 아닌 것은 무엇인가?,3,하,A. 자연어 처리,B. 로봇 공학,C. 웹 디자인,D. 이미지 생성


아래는 Pandas 라이브러리를 사용하여 Python dictionary를 데이터프레임으로 변환하는 코드입니다.


In [39]:
# 데이터프레임을 csv 파일로 저장
df.to_csv("ai_quiz.csv", index=False)

In [40]:
# 데이터프레임을 엑셀 파일로 저장
df.to_excel("ai_quiz.xlsx", index=False)

## 이미지 모드
- 이미지 캡셔닝이랑 비슷하다고 봄.

- input format
```
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])
```

In [42]:
import base64
from openai import OpenAI


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def describe_image(image_path, token_count=False):
    client = OpenAI()

    # 이미지를 base64로 인코딩해야함. 꼭
    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "이 이미지에 대해 자세히 설명해주세요."},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
        max_tokens=2024,
    )
    if token_count:
        return (response.choices[0].message.content, response.usage.total_tokens)
    return response.choices[0].message.content

#### 이미지는 토큰수가 몇개가 될까?
- 이미지는 저해상도(512*512이하)는 85토큰, 고해상도는 170토큰으로 계산한다고 함.
- 그래서 전체 토큰수를 계산하면 다음과 같습니다. 

In [43]:
image_path = "image.png"
(description, token_count) = describe_image(image_path, token_count=True)
print(description, "\ntotal tokens : ", token_count)

이 이미지는 "판다월드 바오 가족 소개"라는 제목이 붙어 있으며, 세 마리의 판다가 등장합니다. 각 판다의 이름과 생년월일이 적혀 있습니다:

1. **야위바오** (영엄): 2013년 7월 13일 태어남
2. **러바오** (안녕): 2012년 7월 28일 태어남
3. **푸바오** (언니): 2020년 7월 20일 태어남

판다들은 자연의 초목과 대나무 배경 속에서 각각의 포즈를 취하고 있습니다. 각 판다는 귀엽고 사랑스러운 표정을 하고 있으며, 기본적인 판다의 특징인 흰색과 검은색의 털을 가지고 있습니다. 전체적으로 따뜻하고 즐거운 분위기를 자아내는 이미지입니다. 
total tokens :  25714


## 함수모드
- 함수모드는 tool을 함수 이름과 함께, 함수가 무엇을 하는지 json으로 정의하고,
- 해당 함수를 정의합니다
- 주석으로 설명해놔야함. 
- tool_choice = auto 로 하면 openai 모델이 알아서 tools을 사용할 것인지 아니면 일반 챗봇 모드가 될 것인지를 결정합니다.  
- 여러개의 tools를 사용할 경우에는 function description이 아주 상세하게 설명을 해야 합니다. OpenAI 모델이 이 function description을 읽고 어떤 tool을 실제로 사용할지를 결정하기 때문입니다. 

In [53]:
# 일단 일반 챗봇모드.


def chat_only(question):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "당신은 친절한 AI Assistance 입니다.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        temperature=0,
    )
    return completion.choices[0].message.content

In [58]:
# tools를 사용한 함수모드

from openai import OpenAI
import datetime
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()


# tools를 사용한 함수모드를 위한 함수 정의
def get_current_datetime():
    "현재 날짜와 시간을 반환하는 함수입니다."

    now = datetime.datetime.now()
    current = now.strftime("%Y-%m-%d %H:%M:%S")
    print("current:", current)
    return current


# 함수모드는 tool 사용.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime", # 함수 이름 정확히 일치해야함 
            #"description": "현재 날짜와 시간을 반환하는 함수입니다.", # 대충써도됨.
            "description": "현재 날짜와 시간을 반환하는 함수입니다. 날짜와 시간을 물어보면 사용해주세요.", # 대충써도됨.
            "parameters": {
                "type": "object",
                "properties": {},  # 함수가 받을 수 있는 파라미터 정의 - get_current_datetime 함수는 파라미터가 없음
                "required": [],  # 필수 파라미터 정의 -  get_current_datetime 함수는 파라미터가 없음
            },
        },
    }
]


def chat_with_tool(user_input):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": user_input}],
        tools=tools, # tools를 사용할 수 있도록 지정!!!!!!!! 
        tool_choice="auto",  # 자동으로 툴 선택 (중요!)
    )

    finished_reason = response.choices[0].finish_reason

    if finished_reason == "tool_calls":

        # 사용 가능한 함수 정의
        available_functions = {
            "get_current_datetime": get_current_datetime,
        }
        # 도구 호출 함수 이름을 가져옵니다.
        function_name = response.choices[0].message.tool_calls[0].function.name
        function_to_call = available_functions[function_name]

        # 함수를 호출하고 응답을 반환합니다.
        function_response = function_to_call()
        return function_response
    else:
        return response.choices[0].message.content


user_query = "오늘이 몇일이지? 지금 몇시야?"
result = chat_with_tool(user_query)
print("tool chat: ", result)

current: 2024-07-31 21:02:50
tool chat:  2024-07-31 21:02:50


In [55]:
# 샘플
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

user_query = "오늘이 몇일이지? 지금 몇시야?"

result = chat_only(user_query)
print("simple chat: ", result)

result = chat_with_tool(user_query)
print("tool chat: ", result)

2024-07-31 20:59:35
simple chat:  지금은 2022년 1월 25일이며, 현재 시간은 사용자의 지역에 따라 다를 수 있습니다. 사용자의 현재 지역을 알려주시면 더 정확한 정보를 제공해 드릴 수 있습니다.
current: 2024-07-31 20:59:39
tool chat:  2024-07-31 20:59:39


In [59]:
user_query = "경상남도 창원에 대해서 설명해주세요"
result = chat_only(user_query)

print("simple chat: ", result)
result = chat_with_tool(user_query)
print("tool chat: ", result)

simple chat:  경상남도 창원은 대한민국 남부에 위치한 도시로, 경상남도의 도청 소재지이기도 합니다. 인구는 약 1백만 명으로 한국에서 4번째로 인구가 많은 도시입니다. 창원은 경제적으로 중요한 도시로서 산업과 무역이 발달하고 있습니다.

창원에는 창원시립박물관, 창원시립미술관, 창원대학교 등 다양한 문화 시설과 교육 기관이 있습니다. 또한 창원은 창원 월영국제영화제와 같은 다양한 문화 행사가 열리는 곳으로도 유명합니다.

또한 창원은 자연환경이 아름다운 곳으로도 유명합니다. 창원에는 창원해양공원, 창원생활과학관, 창원시립수목원 등 다양한 자연 관련 시설이 있어 관광객들에게 인기가 있습니다.

창원은 교통이 편리하고 발전된 도시로서 다양한 산업과 문화를 경험할 수 있는 매력적인 도시입니다.
tool chat:  경상남도 창원은 대한민국의 경상남도에 위치한 도시로, 경상남도의 도청 소재지이자 산업도시입니다. 창원은 다음과 같은 특징이 있습니다.

1. **산업 발전**: 창원은 현대자동차, 두산중공업, LG전자 등 여러 대기업의 생산 기지가 위치해 있으며, 기계, 조선, 전자 산업이 발달해 있습니다. 이로 인해 창원은 "산업의 도시"로 알려져 있습니다.

2. **지리적 위치**: 창원은 남해안에 위치하며, 바다와 가까워 경제적 및 교통적 이점이 있습니다. 주변에는 진해, 마산 등 다른 도시와의 연결이 용이합니다.

3. **교육과 문화**: 창원에는 여러 대학 및 교육 기관이 있으며, 문화 시설도 잘 갖추어져 있습니다. 예를 들어, 창원시립미술관, 창원 문화재단 등이 있습니다.

4. **관광지**: 창원에는 다양한 자연 경관과 관광 명소가 있습니다. 대표적으로 마산만, 진해군항제(벚꽃 축제), 창원 수목원 등이 있습니다.

5. **교통**: 창원은 고속도로와 철도 등 교통망이 잘 발달되어 있어 다른 지역과의 접근성이 좋습니다.

창원은 경제, 문화, 교육 등 다방면에서 활발한 성장과 발전을 이루고 있는 도시입니다.


In [57]:
user_query = "현재 창원의 시간은 무엇인가요?"
result = chat_only(user_query)
print("simple chat: ", result)
result = chat_with_tool(user_query)
print("tool chat: ", result)

simple chat:  죄송합니다, 제가 실시간으로 지역별 시간을 제공할 수는 없습니다. 하지만 창원의 현재 시간을 확인하려면 인터넷 또는 스마트폰의 시계 기능을 이용하시면 됩니다. 혹은 "창원 현재 시간"과 같은 검색어를 사용하여 온라인에서 확인할 수도 있습니다. 다른 도움이 필요하시면 언제든지 물어보세요.
current: 2024-07-31 21:00:54
tool chat:  2024-07-31 21:00:54


=> 현재라는 개념을 simple chat은 모름 but tools을 사용하면 가능 왜냐면 tools에 정의한 함수가 현재 시간이기 때문!